## test

In [ ]:
import py4dgeo


infile1 = r"20_patches_with_seg_id.las"
infile2 = r"21_patches_with_seg_id.las"
epoch0 = py4dgeo.epoch.read_from_las(
    infile1,
    additional_dimensions={"segment_id": "segment_id"},
)

epoch1 = py4dgeo.epoch.read_from_las(
    infile2,
    additional_dimensions={"segment_id": "segment_id"},
)

alg = py4dgeo.PBM3C2(reg_error=0.01)


In [ ]:
import numpy as np
import pandas as pd

segments_dict1 = alg._get_segments(epoch0)
metrics_list_df1 = alg._create_segment_metrics(segments_dict1)

segments_dict2 = alg._get_segments(epoch1)
metrics_list_df2 = alg._create_segment_metrics(segments_dict2)

# use 70% of segments for training, 30% for testing
n_total_1 = len(metrics_list_df1)
n_total_2 = len(metrics_list_df2)
n_train_1 = int(0.7 * n_total_1)
n_train_2 = int(0.7 * n_total_2)

train_metrics_df1 = metrics_list_df1.iloc[:n_train_1].copy()
train_metrics_df2 = metrics_list_df2.iloc[:n_train_2].copy()

train_metrics_df1 = train_metrics_df1.sample(frac=1)
train_metrics_df2 = train_metrics_df2.sample(frac=1)

train_metrics_df1 = train_metrics_df1.reset_index()
train_metrics_df2 = train_metrics_df2.reset_index()

n_pairs = min(len(train_metrics_df1), len(train_metrics_df2))
train_metrics_df1 = train_metrics_df1.iloc[:n_pairs]
train_metrics_df2 = train_metrics_df2.iloc[:n_pairs]

print("train_metrics_df1 columns:", train_metrics_df1.columns.tolist())
print("train_metrics_df2 columns:", train_metrics_df2.columns.tolist())

joined_df = pd.DataFrame({
    'segment_id_epoch0': train_metrics_df1['segment_id'].values,
    'segment_id_epoch1': train_metrics_df2['segment_id'].values,
})

joined_df['is_matching'] = np.random.choice([0, 1], size=len(joined_df))

joined_df = joined_df.dropna()
joined_df.to_csv(r"joined_segments.csv", index=False, header=False)

print(f"Total segments in epoch0: {n_total_1}")
print(f"Total segments in epoch1: {n_total_2}")
print(f"Training set size (70%): {len(joined_df)}")
print(f"Test set size (30% of epoch0): {n_total_1 - n_train_1}")

test_metrics_df1 = metrics_list_df1.iloc[n_train_1:].copy()
apply_ids = test_metrics_df1.index.values  

print(f"Apply IDs (first 10): {apply_ids[:10]}")

In [ ]:
epoch0_seg_ids = np.unique(epoch0.additional_dimensions["segment_id"])
epoch0_seg_id_range = (epoch0_seg_ids.min(), epoch0_seg_ids.max())

print(f"Epoch0 actual segment_id range: {epoch0_seg_id_range}")
print(f"Epoch0 unique segment count: {len(epoch0_seg_ids)}")

epoch1_seg_ids = np.unique(epoch1.additional_dimensions["segment_id"])
epoch1_seg_id_range = (epoch1_seg_ids.min(), epoch1_seg_ids.max())

print(f"Epoch1 actual segment_id range: {epoch1_seg_id_range}")
print(f"Epoch1 unique segment count: {len(epoch1_seg_ids)}")

In [ ]:
correspondences_df = alg.compute(
    epoch0=epoch0,
    epoch1=epoch1,
    correspondences_file="joined_segments.csv",
    apply_ids=apply_ids,
    search_radius=5.0,
)

In [ ]:
print(correspondences_df)

distances = correspondences_df["distance"]
uncertainties = correspondences_df["uncertainty"]